In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
from multiprocessing.dummy import Pool as ThreadPool
import time
import re
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

import os

import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='ru')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.metrics import f1_score

import random

In [142]:
doc_to_title = {}
with open('docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print(len(doc_to_title))

28026


In [143]:
import pandas as pd
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))
type(train_data)

pandas.core.frame.DataFrame

In [144]:
import numpy as np
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:25]    )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11690, 25) (11690,) (11690,)


In [145]:
X_train.max()

22

In [146]:
def get_topk_inter_dist(texts, k=25):
    X = np.zeros(shape=(len(texts), len(texts)), dtype=float)
    for i in range(len(texts)):
        words_i = set(texts[i].lower().strip().split())
        for j in range(i + 1, len(texts)):
            words_j = set(texts[j].lower().strip().split())
            X[i, j] = X[j, i] = len(words_i & words_j)
    return np.sort(X, axis=1)[:, :-k-1:-1]

In [147]:
def get_topk_cosine_dist(texts, ngrams=(1, 1), k=25):
    vectorizer = TfidfVectorizer(ngram_range=ngrams)
    X = vectorizer.fit_transform(texts).toarray()
    return np.sort(pairwise_distances(X, X, metric='cosine'), axis=1)[:, 1:k+1]

In [148]:
train_data

,pair_id,group_id,doc_id,target
0,1,1,15731,0
1,2,1,14829,0
2,3,1,15764,0
3,4,1,17669,0
4,5,1,14852,0
...,...,...,...,...
11685,11686,129,26672,0
11686,11687,129,25838,0
11687,11688,129,25703,0
11688,11689,129,27885,0


In [161]:
#pair_id, group_id, doc_id, target
def get_features(data, k=25):
    X = np.ndarray(shape=(0, 2 * k))
    y = []
    for gr_id in data.group_id.unique():
        texts = []
        if 'target' in data.columns:

            for _, _, doc_id, target in data[data.group_id == gr_id].values:
                y.append(target)
                with open('data_title_h13/' + str(doc_id) + '.dat.txt', mode='r') as doc:
                    texts.append(doc.read())
        else:
            for _, _, doc_id in data[data.group_id == gr_id].values:
                with open('data_title_h13/' + str(doc_id) + '.dat.txt', mode='r') as doc:
                    texts.append(doc.read())
        feat_gr = np.hstack((get_topk_inter_dist(texts, k), get_topk_cosine_dist(texts, k=k)))
        X = np.vstack((X, feat_gr))
    if 'target' in data.columns:
        return X, np.asarray(y)
    else:
        return X

In [162]:
shuff_ids = list(train_data.group_id)
random.shuffle(shuff_ids)
train_groups, test_groups = shuff_ids[:110], shuff_ids[110:]

In [163]:
train = train_data.loc[train_data.group_id.isin(train_groups)]

In [164]:
train

,pair_id,group_id,doc_id,target
0,1,1,15731,0
1,2,1,14829,0
2,3,1,15764,0
3,4,1,17669,0
4,5,1,14852,0
...,...,...,...,...
11544,11545,127,21803,0
11545,11546,127,24015,1
11546,11547,127,22439,0
11547,11548,127,23312,0


In [178]:
X_train, y_train = get_features(train)
X_val, y_val = get_features(train_data.loc[train_data.group_id.isin(test_groups)])

In [166]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedKFold, KFold

In [179]:
X_train.shape

(6662, 50)

In [180]:
clf = lgb.LGBMClassifier(max_depth=6, reg_lambda=1.0)

In [181]:
clf.fit(X_train, y_train)

LGBMClassifier(max_depth=6, reg_lambda=1.0)

In [182]:
y_pred = clf.predict(X_val)

In [183]:
f1_score(y_val, y_pred)

0.7498785818358427

In [184]:
doc_to_title = {}
with open('docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print(len(doc_to_title))

28026


In [185]:
import pandas as pd
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))
type(train_data)

pandas.core.frame.DataFrame

In [186]:
test_data = pd.read_csv('test_groups.csv')
testgroups_titledata = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))
test_data.head()

,pair_id,group_id,doc_id
0,11691,130,6710
1,11692,130,4030
2,11693,130,5561
3,11694,130,4055
4,11695,130,4247


In [187]:
X_train, y_train = get_features(train_data)
X_val = get_features(test_data)

In [188]:
clf = lgb.LGBMClassifier(max_depth=6, reg_lambda=1.0)
clf.fit(X_train, y_train)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


LGBMClassifier(max_depth=6, reg_lambda=1.0)

In [189]:
y_pred = clf.predict(X_val)

In [190]:
data = {'pair_id': np.asarray(test_data.pair_id), 'target': y_pred}
df = pd.DataFrame(data=data)
df = df.set_index(keys=['pair_id'])
df.to_csv('submitBoostHeaders.csv')
df.head()

,target
pair_id,
11691,1
11692,1
11693,1
11694,1
11695,0


In [191]:
print((y_pred == 1).astype(int).sum())
print((y_pred == 0).astype(int).sum())

print((y_train == 1).astype(int).sum())
print((y_train == 0).astype(int).sum())

4209
12418
3361
8329


скор на лидерборде: 0.69654